**RDF creation of tensile test data using TTO**

In this notebook, data originating from tensile tests performed to an S355 steel sheet is read in to create an exemplary RDF data file that uses the [Tensile Test Ontology (TTO) version 3.0](https://github.com/materialdigital/tensile-test-ontology) as semantic backbone. Hence, the tensile test data available in 'classic' test formats is transformed to RDF data by creating triples using the RDFlib library and some helper functions. Therefore, values tokenized from the CSV are allocated to concepts of the TTO developed in the frame of the PMD project. Since the TTO is strongly based on the PMD Core Ontology - PMDco (namespace: [https://w3id.org/pmd/co](https://w3id.org/pmd/co)) - the PMDco will also be parsed and semantic concepts this ontology is related to will also be used, accordingly. The tensile test data is taken from a [dataset published openly available on Zenodo](https://zenodo.org/record/6778336).
The RDF data is directly saved in a graph. Subsequently, the graph is serialized and saved to a TTL and an RDF file. Furthermore, SPARQL queries may be performed for consistency checks.
Hence, all in all, this script is used to create a knowledge graph (A-Box) using vocabulary of several ontologies and tensile test data originating from real tests performed on an S355 steel in accordance with the corresponding test standard [ISO 6892-1:2019-11](https://www.beuth.de/de/norm/iso-6892-1/316885984).

# Installing and Importing relevant Python packages

In [ ]:
%%capture
%pip install --upgrade pip
%pip install rdflib

In [ ]:
# Import of relevant package parts
from rdflib import Graph, Literal, URIRef, BNode, Namespace, DC
from rdflib.term import Identifier
from rdflib.collection import Collection
from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL
import rdflib.plugins.sparql.update
import pandas as pd
import io
import urllib.parse

# Definition of helper methods

In [16]:
from rdflib import URIRef, Literal, XSD
# Definition of helper methods / functions - do not necessarily have to be used.

# Helper method 'add' used to write triples to an RDF graph
def add(s,p,o):

    # in this case p is "ObjectProperty"
    if o.find('http://')==0 or o.find('https://')==0:
        g.add( (URIRef(s), URIRef(p), URIRef(o)) )

    # in this case p is "DatatypeProperty"
    else:
        # if we can parse o as Float, just set the datatype
        try:
            g.add( (URIRef(s), URIRef(p), Literal(float(o), datatype=XSD.float) ))
        except:
            g.add( (URIRef(s), URIRef(p), Literal(o) ))


# Global counter possibly usable for the creation of arbitrary instance names (numbers)
instanceCounter = 0
def nextInstanceNum():
    global instanceCounter
    instanceCounter = instanceCounter + 1
    return str(instanceCounter)


# Turtle style abbreviation for RDF.type
a = RDF.type


# Function for adding new entries to the A-Box representing a quality including scalar value specification, scalar measurement datum, and process / testpiece designation.
# Since it is a scalar value measurement representation, values are set as xsd.float !
def add_scalar_value_measurement(
    g,
    experimentIRI,
    measurement_name,        # z.B. "thickness_a1"
    value,                   # z.B. a1
    unit,                    # z.B. unit.MilliM, if there is no unit to be assigned, this has to be given as argument by "None"
    quality_class,           # z.B. base.TTO_0000029
    testpieceIRI,
    processIRI,
    processPartIRI = None    # optional, Default = None
):
    """
    Inserts a measurement (Quality, Scalar Value Spec, Scalar Measurement Datum)
    into the ABox.
    """
    # Quality
    qualityIRI = URIRef(experimentIRI + f"_{measurement_name}_quality") # creation of a quality regarded (definition: s.a., given by method argument "measurement_name")
    g.add((qualityIRI, a, quality_class)) # is a quality class (definition: s.a., given by method argument) as subclass of PMD:quality
    g.add((testpieceIRI, obo.BFO_0000196, qualityIRI)) # Testpiece BFO:bearer of quality just defined

    # Scalar Value Specification
    svsIRI = URIRef(experimentIRI + f"_{measurement_name}_scalar_value_specification") # creation of a respective scalar value specification, referring to above-created quality
    g.add((svsIRI, a, pmdco.PMD_0000022)) # scalar measurement value specification is a PMD:scalar value specification
    g.add((svsIRI, pmdco.PMD_0060001, qualityIRI)) # scalar measurement value specification PMD:specifies value of quality defined above
    g.add((svsIRI, obo.IAO_0000136, testpieceIRI)) # scalar measurement value specification IAO:is about test piece
    g.add((svsIRI, pmdco.PMD_0000006, Literal(value, datatype=XSD.float))) # scalar value specification PMD:has value value as defined above (given by method argument "value", is usually a variable)
    if unit is not None:
        g.add((svsIRI, pmdco.PMD_0000020, unit)) # scalar value specification PMD:has measurement unit label unit as defined above (given by method argument "unit", is usually a QUDT term)
 
    # Scalar Measurement Datum
    smdIRI = URIRef(experimentIRI + f"_{measurement_name}_scalar_measurement_datum") # creation of a respective scalar measurement datum referring to above-created quality
    g.add((smdIRI, a, pmdco.PMD_0000023)) # is a PMD:scalar measurement datum as subclass of IAO:data item (IAO:information content entity)
    g.add((qualityIRI, obo.IAO_0000417, smdIRI)) # quality IAO:is quality measured as scalar measurement datum
    g.add((smdIRI, pmdco.PMD_0060000, svsIRI)) # scalar measurement datum PMD:has scalar value specification scalar measurement value specification

    # Process Linkages
    g.add((processIRI, pmdco.PMD_0000016, qualityIRI)) # process PMD:has output quality (being a subclass of BFO:quality)
    if processPartIRI is not None:
        g.add((processPartIRI, pmdco.PMD_0000016, qualityIRI)) # if there is a part process defined, also this part process PMD:has output quality (being a subclass of BFO:quality)

    # return qualityIRI, svsIRI, smdIRI # may be used for later; originally, there are is no value return needed
    
    
    
# Function for adding new entries to the A-Box representing a quality as metadata to a process.
# Since it is a scalar value measurement representation, values are set as xsd.float !
def add_scalar_value_measurement_metadata(
    g,
    experimentIRI,
    measurement_name,        # z.B. "environmental_temperature_T"
    value,                   # z.B. T
    unit,                    # z.B. unit.DEG_C, if there is no unit to be assigned, this has to be given as argument by "None"
    quality_class,           # z.B. pmdco.PMD_0000967
    processIRI = None,       # optional, Default = None
    processPartIRI = None    # optional, Default = None
):
    """
    Inserts a measurement (Quality, Scalar Value Spec, Scalar Measurement Datum) as metadata
    into the ABox.
    """
    # Quality
    qualityIRI = URIRef(experimentIRI + f"_{measurement_name}_quality") # creation of a quality regarded (definition: s.a., given by method argument "measurement_name")
    g.add((qualityIRI, a, quality_class)) # is a quality class (definition: s.a., given by method argument) as subclass of PMD:quality

    # Scalar Value Specification
    svsIRI = URIRef(experimentIRI + f"_{measurement_name}_scalar_value_specification") # creation of a respective scalar value specification, referring to above-created quality
    g.add((svsIRI, a, pmdco.PMD_0000022)) # scalar measurement value specification is a PMD:scalar value specification
    g.add((svsIRI, pmdco.PMD_0060001, qualityIRI)) # scalar measurement value specification PMD:specifies value of quality defined above
    g.add((svsIRI, pmdco.PMD_0000006, Literal(value, datatype=XSD.float))) # scalar value specification PMD:has value value as defined above (given by method argument "value", is usually a variable)
    if unit is not None:
        g.add((svsIRI, pmdco.PMD_0000020, unit)) # scalar value specification PMD:has measurement unit label unit as defined above (given by method argument "unit", is usually a QUDT term)
 
    # Scalar Measurement Datum
    smdIRI = URIRef(experimentIRI + f"_{measurement_name}_scalar_measurement_datum") # creation of a respective scalar measurement datum referring to above-created quality
    g.add((smdIRI, a, pmdco.PMD_0000023)) # is a PMD:scalar measurement datum as subclass of IAO:data item (IAO:information content entity)
    g.add((qualityIRI, obo.IAO_0000417, smdIRI)) # quality IAO:is quality measured as scalar measurement datum
    g.add((smdIRI, pmdco.PMD_0060000, svsIRI)) # scalar measurement datum PMD:has scalar value specification scalar measurement value specification

    # Process Linkages
    if processIRI is not None:
        g.add((processIRI, obo.BFO_0000057, qualityIRI)) # process BFO:has participant quality (being a subclass of BFO:quality), because it is metadata
    if processPartIRI is not None:
        g.add((processPartIRI, obo.BFO_0000057, qualityIRI)) # if there is a part process defined, also this part process BFO:has participant quality (being a subclass of BFO:quality), because it is metadata

# Function for adding new entries to the A-Box representing a project as metadata to a process.
def add_project(g, experimentIRI, processIRI, project_literal):
    projectIRI = URIRef(experimentIRI + "_project")
    g.add((projectIRI, a, pmdco.PMD_0000909))  # is a pmdco:project as subclass BFO:process
    g.add((projectIRI, obo.BFO_0000117, processIRI)) # Project BFO:has occurrent part process as defined beforehand

    projectIdentifierIRI = URIRef(experimentIRI + "_project_ID")
    g.add((projectIdentifierIRI, a, pmdco.PMD_0000017))  # is a PMD:identifier as subclass of IAO:information content entity
    g.add((projectIdentifierIRI, obo.IAO_0000219, projectIRI)) # (project) identifier IAO:denotes project

    projectIdentifierValueIRI = URIRef(experimentIRI + "_project_ID_value")
    g.add((projectIdentifierValueIRI, a, obo.OBI_0001933))  # is a OBI:value specification as subclass of IAO:information content entity
    g.add((projectIdentifierValueIRI, pmdco.PMD_0000006, Literal(project_literal, datatype=XSD.string))) # Project identifier value PMD:has value Literal (project_literal) as given in argument
    g.add((projectIdentifierIRI, pmdco.PMD_0060000, projectIdentifierValueIRI)) # Project identifier PMD:has value specification project identifier value

    return projectIRI

# Function for adding new entries to the A-Box representing a (project) funder as metadata to a process.
def add_funder(g, experimentIRI, projectIRI, funder_literal_de = None, funder_literal_en = None, funder_number_literal = None):
    funderRoleIRI = URIRef(experimentIRI + "_funder_role")
    g.add((funderRoleIRI, a, base.TTO_0000000)) # is a base:funder role (here, TTO:funder role as TTO_0000000)
    g.add((funderRoleIRI, obo.BFO_0000054, projectIRI)) # Funder role BFO:has realization (in) project

    funderIRI = URIRef(experimentIRI + "_funder")
    g.add((funderIRI, a, pmdco.PMD_0000879))  # is a PMD:organization as subclass of material entity (inndependent continuant)
    g.add((funderIRI, obo.BFO_0000196, funderRoleIRI)) # Funder BFO:bearer of funder role
    g.add((projectIRI, obo.BFO_0000057, funderIRI)) # Project BFO:has participant funder

    # Name identifier
    funderNameIRI = URIRef(experimentIRI + "_funder_ID_name")
    g.add((funderNameIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
    g.add((funderNameIRI, obo.IAO_0000219, funderIRI)) # Funder Name (identifier) iao:denotes Funder

    funderNameValueIRI = URIRef(experimentIRI + "_funder_name")
    g.add((funderNameValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
    if funder_literal_de is not None:
        g.add((funderNameValueIRI, pmdco.PMD_0000006, Literal(funder_literal_de, datatype=XSD.string))) # Funder identifier value PMD:has value "Name" given as string in German
    if funder_literal_en is not None:
        g.add((funderNameValueIRI, pmdco.PMD_0000006, Literal(funder_literal_en, datatype=XSD.string))) # Funder identifier value PMD:has value "Name" given as string in English
    g.add((funderNameIRI, pmdco.PMD_0060000, funderNameValueIRI)) # Funder identifier PMD:has value specification funder identifier value

    # Number identifier
    funderNumberIRI = URIRef(experimentIRI + "_funder_ID_number")
    g.add((funderNumberIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
    g.add((funderNumberIRI, obo.IAO_0000219, funderIRI)) # Funder number (identifier) iao:denotes Funder

    funderNumberValueIRI = URIRef(experimentIRI + "_funder_number")
    g.add((funderNumberValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
    if funder_number_literal is not None:
        g.add((funderNumberValueIRI, pmdco.PMD_0000006, Literal(funder_number_literal, datatype=XSD.string))) # Funder identifier value PMD:has value "Number" (as string, though) given by the argument
    g.add((funderNumberIRI, pmdco.PMD_0060000, funderNumberValueIRI)) # Funder identifier PMD:has value specification funder identifier value

    # return funderIRI
    
# Function for adding new entries to the A-Box representing a funder project (which might be an umbrella project to the one regarded directly) as metadata to a process.
def add_funding_project(g, experimentIRI, projectIRI, funding_project_literal):
    fundingProjectIRI = URIRef(experimentIRI + "_funding_project")
    g.add((fundingProjectIRI, a, pmdco.PMD_0000909)) # is a pmdco:project as subclass BFO:process
    g.add((fundingProjectIRI, obo.BFO_0000117, projectIRI)) # Project BFO:has occurrent part project

    fundingProjectIdentifierIRI = URIRef(experimentIRI + "_funding_project_ID")
    g.add((fundingProjectIdentifierIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier as subclass of IAO:information content entity
    g.add((fundingProjectIdentifierIRI, obo.IAO_0000219, fundingProjectIRI)) # (funding project) identifier IAO:denotes funding project

    fundingProjectIdentifierValueIRI = URIRef(experimentIRI + "_funding_project_ID_value")
    g.add((fundingProjectIdentifierValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
    g.add((fundingProjectIdentifierValueIRI, pmdco.PMD_0000006, Literal(funding_project_literal, datatype=XSD.string))) # Funding project identifier value PMD:has value Literal given as string within the argument of the function
    g.add((fundingProjectIdentifierIRI, pmdco.PMD_0060000, fundingProjectIdentifierValueIRI)) # Funding project identifier PMD:has value specification funding project identifier value

    # return fundingProjectIRI
    
    
# Function for adding new entries to the A-Box representing a device
def add_test_device(g, experimentIRI, processIRI, deviceDenotation, deviceClass, functionClass, processPart1IRI=None, processPart2IRI=None, deviceName=None, deviceManufacturer=None, deviceType=None, deviceSerialNumber=None, deviceStandard=None):
    # Machine Function
    machineFunctionIRI = URIRef(experimentIRI + f"_{deviceDenotation}_function")
    g.add((machineFunctionIRI, a, functionClass)) # is a functionClass (variable, given in the arguments)
    g.add((machineFunctionIRI, obo.BFO_0000054, processIRI)) # (tensile) testing function BFO:has realization (in) (tensile testing) process
    if processPart1IRI is not None:
        g.add((machineFunctionIRI, obo.BFO_0000054, processPart1IRI)) # (tensile) testing function BFO:has realization (in) subprocess
    if processPart2IRI is not None:
        g.add((machineFunctionIRI, obo.BFO_0000054, processPart2IRI)) # (tensile) testing function BFO:has realization (in) subprocess

    # Machine
    machineIRI = URIRef(experimentIRI + f"_{deviceDenotation}")
    g.add((machineIRI, a, deviceClass)) # Machine is a deviceClass (variable, given in the arguments)
    g.add((machineIRI, obo.BFO_0000196, machineFunctionIRI)) # Test device BFO:bearer of testing function
    g.add((processIRI, obo.BFO_0000057, machineIRI)) # Process (tensile testing) BFO:has participant Device
    if processPart1IRI is not None:
        g.add((processPart1IRI, obo.BFO_0000057, machineIRI)) # ISO Process (tensile testing subprocess) BFO:has participant Device
    if processPart2IRI is not None:
        g.add((processPart2IRI, obo.BFO_0000057, machineIRI)) # ISO Process (tensile testing subprocess) BFO:has participant Device

    # Machine identifiers (optional)
    if deviceName is not None:
        machineNameIRI = URIRef(experimentIRI + f"_{deviceDenotation}_ID_name")
        g.add((machineNameIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
        g.add((machineNameIRI, obo.IAO_0000219, machineIRI)) # Machine Name (identifier) iao:denotes machine

        machineNameValueIRI = URIRef(experimentIRI + f"_{deviceDenotation}_ID_name_value")
        g.add((machineNameValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
        g.add((machineNameValueIRI, pmdco.PMD_0000006, Literal(deviceName, datatype=XSD.string))) # Machine identifier value PMD:has value machineName (variable)
        g.add((machineNameIRI, pmdco.PMD_0060000, machineNameValueIRI)) # Machine name (identifier) PMD:has value specification machine name value

    # Device manufacturer identification (optional)
    if deviceManufacturer is not None:
        machineManufacturerRoleIRI = URIRef(experimentIRI + f"_{deviceDenotation}_manufacturer_role")
        g.add((machineManufacturerRoleIRI, a, pmdco.PMD_0000831)) # is a Manufacturer Role which is a subclass of role --> realizable entity (in OBI)

        machineManufacturerIRI = URIRef(experimentIRI + f"_{deviceDenotation}_manufacturer")
        g.add((machineManufacturerIRI, a, pmdco.PMD_0000830)) # is a PMD:Manufacturer company which is a subclass of Organization --> material entity
        g.add((machineIRI, obo.BFO_0000196, machineManufacturerRoleIRI)) # The Manufacturer company BFO:bearer of Manufacturer role
        g.add((machineIRI, pmdco.PMD_0000178, machineManufacturerIRI)) # Device PMD:has continuant part (tensile testing) machine manufacturer company

        machineManufacturerNameIRI = URIRef(experimentIRI + f"_{deviceDenotation}_manufacturer_ID_name")
        g.add((machineManufacturerNameIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
        g.add((machineManufacturerNameIRI, obo.IAO_0000219, machineManufacturerIRI)) # Machine Manufacturer Name (identifier) iao:denotes machine manufacturer

        machineManufacturerNameValueIRI = URIRef(experimentIRI + f"_{deviceDenotation}_manufacturer_ID_name_value")
        g.add((machineManufacturerNameValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
        g.add((machineManufacturerNameValueIRI, pmdco.PMD_0000006, Literal(deviceManufacturer, datatype=XSD.string))) # Machine manufacturer name value PMD:has value deviceManufacturer (variable)
        g.add((machineManufacturerNameIRI, pmdco.PMD_0060000, machineManufacturerNameValueIRI)) # Machine manufacturer name (identifier) PMD:has value specification Machine manufacturer name value

    # Machine type (optional)
    if deviceType is not None:
        machineTypeIRI = URIRef(experimentIRI + f"_{deviceDenotation}_type")
        g.add((machineTypeIRI, a, obo.IAO_0020000)) # is an iao:identifier
        g.add((machineTypeIRI, obo.IAO_0000219, machineIRI)) # Machine type (identifier) iao:denotes Device

        machineTypeValueIRI = URIRef(experimentIRI + f"_{deviceDenotation}_type_value")
        g.add((machineTypeValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
        g.add((machineTypeValueIRI, pmdco.PMD_0000006, Literal(deviceType, datatype=XSD.string))) # Machine type value PMD:has value deviceType (variable)
        g.add((machineTypeIRI, pmdco.PMD_0060000, machineTypeValueIRI)) # Machine type (identifier) PMD:has value specification Machine type value

    # Machine serial number (optional)
    if deviceSerialNumber is not None:
        machineSerialNumberIRI = URIRef(experimentIRI + f"_{deviceDenotation}_serial_number")
        g.add((machineSerialNumberIRI, a, obo.IAO_0020000)) # is an iao:identifier --> iao:information content entity
        g.add((machineSerialNumberIRI, obo.IAO_0000219, machineIRI)) # Machine serial number (identifier) iao:denotes Device

        machineSerialNumberValueIRI = URIRef(experimentIRI + f"_{deviceDenotation}_serial_number_value")
        g.add((machineSerialNumberValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
        g.add((machineSerialNumberValueIRI, pmdco.PMD_0000006, Literal(deviceSerialNumber, datatype=XSD.string))) # Machine serial number value PMD:has value deviceSerialNumber (variable)
        g.add((machineSerialNumberIRI, pmdco.PMD_0060000, machineSerialNumberValueIRI)) # Machine serial number (identifier) PMD:has value specification Machine serial number value

    # Machine standard (optional)
    if deviceStandard is not None:
        machineStandardIRI = URIRef(experimentIRI + f"_{deviceDenotation}_standard")
        g.add((machineStandardIRI, a, pmdco.PMD_0060006)) # is a pmdco:DeviceSpecification as subclass of iao:directive information entity --> iao:information content entity
        g.add((machineStandardIRI, obo.IAO_0000219, machineIRI)) # Machine standard (norm) (identifier) iao:denotes Device

        machineStandardValueIRI = URIRef(experimentIRI + f"_{deviceDenotation}_standard_value")
        g.add((machineStandardValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
        g.add((machineStandardValueIRI, pmdco.PMD_0000006, Literal(deviceStandard, datatype=XSD.string))) # Machine standard value PMD:has value deviceStandard (variable)
        g.add((machineStandardIRI, pmdco.PMD_0060000, machineStandardValueIRI)) # Machine standard (identifier) PMD:has value specification Machine standard value

    # return machineIRI

# Read in tensile test data

In [ ]:
# Read in data that is supposed to be mapped to the concepts of the ontology (TTO).
# The data is also findable as a dataset on tensile tests that is published freely available on Zenodo: https://zenodo.org/record/6778336

# Reading in data from URL
link_data = "https://raw.githubusercontent.com/materialdigital/tensile-test-ontology/refs/heads/main/tensile_test_data/S355_data.csv"
data_S333_read_in = pd.read_csv(link_data, sep=';')

# Graph creation using RDFlib

In [18]:
# Creating the RDF graph 'g'
g = Graph()

In [19]:
# Definition of IRI prefixes (namespaces)
base = Namespace("https://w3id.org/pmd/tto/")
g.bind("base", base)
pmdco = Namespace("https://w3id.org/pmd/co/")
g.bind("pmdco", pmdco)
unit = Namespace("http://qudt.org/vocab/unit/")
g.bind("unit", unit)
prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)
raw_data = Namespace("https://github.com/materialdigital/tensile-test-ontology/tree/3dc72e40d492402e5ccd46167b8c03c62599c5dc/data/primary_data/")
g.bind("raw_data", raw_data)
csvw = Namespace("http://www.w3.org/ns/csvw#")
g.bind("csvw", csvw)
datacite = Namespace("http://purl.org/spar/datacite/")
g.bind("datacite", datacite)
obo = Namespace("http://purl.obolibrary.org/obo/")
g.bind("obo", obo)
dce = Namespace("http://purl.org/dc/elements/1.1/")
g.bind("dce", dce)
time = Namespace("http://www.w3.org/2006/time#")
g.bind("time", time)
dct = Namespace("http://purl.org/dc/terms/")
g.bind("dct", dct)

# A-Box namespace (namespace of instances)
prefix = Namespace("https://w3id.org/pmd/demodata/tensiletest_S355/")
g.bind("prefix", prefix)

# Turtle style abbreviation for RDF.type
a = RDF.type

# Create ontology and import TTO as basis
onto = URIRef(prefix)
g.add((onto, a, OWL.Ontology))
g.add((onto, OWL.imports, URIRef(base)))

# Add creators, title, version number, short description
markus = URIRef("https://orcid.org/0000-0002-7094-5371")
bernd = URIRef("https://orcid.org/0000-0002-3717-7104")
joerg = URIRef("https://orcid.org/0000-0001-7192-7143")
philipp = URIRef("https://orcid.org/0000-0003-4971-3645")
thomas = URIRef("https://orcid.org/0000-0003-1649-6832")
kamilla = URIRef("https://orcid.org/0009-0003-1503-4422")

g.add((onto, dct.creator, markus))
g.add((markus, RDFS.label, Literal("Markus Schilling", datatype=XSD.string)))
g.add((onto, dct.creator, bernd))
g.add((bernd, RDFS.label, Literal("Bernd Bayerlein", datatype=XSD.string)))
g.add((onto, dct.creator, joerg))
g.add((joerg, RDFS.label, Literal("Jörg Waitelonis", datatype=XSD.string)))
g.add((onto, dct.creator, philipp))
g.add((philipp, RDFS.label, Literal("Philipp von Hartrott", datatype=XSD.string)))
g.add((onto, dct.creator, thomas))
g.add((thomas, RDFS.label, Literal("Thomas Hanke", datatype=XSD.string)))
g.add((onto, dct.creator, kamilla))
g.add((kamilla, RDFS.label, Literal("Kamilla Zaripova", datatype=XSD.string)))

g.add((onto, DC.title, Literal("Tensile Test Ontology (TTO) A-Box Data Mapping Example", datatype=XSD.string)))
g.add((onto, OWL.versionInfo, Literal("3.0.0", datatype=XSD.string)))
g.add((onto, dct.description, Literal("This is an exemplary A-Box (instance data) representing tensile test results performed on an S355 steel sheet according to ISO 6892-1:2019-11. The data originates from a publicly available dataset hosted on Zenodo: https://zenodo.org/record/6778336 . The semantic structure is based on the Tensile Test Ontology (TTO) version 3.0 (https://github.com/materialdigital/tensile-test-ontology), complemented by concepts from the PMD Core Ontology (PMDco).", datatype=XSD.string)))

# Load data from read in above.
data = data_S333_read_in

# Creation of QUDT unit instances as instances of iao:measurement unit label
# Class "mesurement unit label" has IRI: obo.IAO_0000003 (see below for more details)
g.add((unit.MilliM, a, obo.IAO_0000003))
g.add((unit.SEC, a, obo.IAO_0000003))
g.add((unit.KiloN, a, obo.IAO_0000003))
g.add((unit.PERCENT, a, obo.IAO_0000003))
g.add((unit.MilliM2, a, obo.IAO_0000003))
g.add((unit.MegaPa, a, obo.IAO_0000003))
g.add((unit.GigaPa, a, obo.IAO_0000003))
g.add((unit.DEG_C, a, obo.IAO_0000003))

# Creation of triples
# Iterate over the full data table
for idx, row in data.iterrows():
  # Definition of tokens in accordance with columns of original csv file
  process_id                          = row["process"]
  testpiece_id                        = row["testpiece"]
  a1                                  = row["d1 / a1"]
  a2                                  = row["d2 / a2"]
  a3                                  = row["d3 / a3"]
  b1                                  = row["b1"]
  b2                                  = row["b2"]
  b3                                  = row["b3"]
  s1                                  = row["s1"]
  s2                                  = row["s2"]
  s3                                  = row["s3"]
  S0                                  = row["S0"]
  Rp02                                = row["Rp0,2"]
  ReH                                 = row["ReH"]
  Fm                                  = row["Fm"]
  Rm                                  = row["Rm"]
  E                                   = row["E"]
  L0                                  = row["L0"]
  Lu                                  = row["Lu"]
  A                                   = row["A"]
  au                                  = row["au"]
  bu                                  = row["bu"]
  Su                                  = row["Su"]
  Z                                   = row["Z"]
  operator                            = row["Operator"]
  date                                = row["date"]
  materialDescription                 = row["Werkstoff"]
  specimenShape                       = row["Probenform"]
  note                                = row["Note"]
  environmentalTemperature            = row["Umgebungstemperatur"]
  machineName                         = row["Prüfmaschinen-Name"]
  machineManufacturer                 = row["Prüfmaschinenhersteller"]
  machineType                         = row["Prüfmaschinen-Typ"]
  machineSerialNumber                 = row["Prüfmaschinen-Seriennummer"]
  machineStandard                     = row["Prüfmaschinen-Standard"]
  extensometerName                    = row["Extensometer"]
  extensometerSerialNumber            = row["Extensometer-Seriennummer"]
  extensometerStandard                = row["Extensometer-Standard"]
  micrometerGaugeManufacturer         = row["Mikrometerschraube-Hersteller"]
  caliperManufacturer                 = row["Messschieber-Hersteller"]
  caliperSerialNumber                 = row["Messschieber-Seriennummer"]
  strainRate                          = row["Dehngeschwindigkeit"]
  transitionPoint                     = row["Umschaltpunkt"]
  extensometerGaugeLength             = row["Extensometer-Nenngerätemesslänger"]
  loadCellMaxForce                    = row["Kraftmessdose-Kraftbereich"]
  project                             = row["Project"]
  standard                            = row["Norm"]
  rollingDirection                    = row["Walzrichtung"]
  location                            = row["Ort"]
  institute                           = row["Institut"]
  address                             = row["Adresse"]
 
  # Creation of triples for all instances (considering all connections between them) based on a bibliographic approach (RDFlib). There is about one "block" per instance.
  experimentIRI = URIRef(prefix + process_id)
  g.add((experimentIRI, a, pmdco.PMD_0000017)) # PMD:identifier
  g.add((experimentIRI, RDFS.label, Literal(process_id, datatype=XSD.string)))  

  processIRI = URIRef(experimentIRI + "_process")
  g.add((processIRI, a, pmdco.PMD_0000974)) # PMD:tensile testing process as subclass of PMD:assay (BFO:process)
  g.add((experimentIRI, obo.IAO_0000219, processIRI)) # Experiment ID denotes Process (Tensile Test)

  # Creation Process Plan Specification Instance "Tensile Test Method"
  processPlanIRI = URIRef(experimentIRI + "_tensile_test_method")
  g.add((processPlanIRI, a, base.TTO_0000054)) # tto:tensile test method as subclass of PMD:plan specification (BFO:realizable entity)
  g.add((processIRI, obo.BFO_0000055, processPlanIRI)) # (Tensile Test) Process bfo:realizes the process plan, named tensile test method

  # Test Standard as identifier of "tensile test method" plan specification
  # Name identifier
  processPlanIdentifierIRI = URIRef(experimentIRI + "_tensile_test_method_ID")
  g.add((processPlanIdentifierIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
  g.add((processPlanIdentifierIRI, obo.IAO_0000219, processPlanIRI)) # Tensile test method identifier (Name) IAO:denotes process plan specification (realizable entity) named tensile test method

  processPlanIdentifierValueIRI = URIRef(experimentIRI + "_tensile_test_method_ID_standard")
  g.add((processPlanIdentifierValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
  g.add((processPlanIdentifierValueIRI, pmdco.PMD_0000006, Literal(standard, datatype=XSD.string))) # Process plan identifier value PMD:has value standard (variable)
  g.add((processPlanIdentifierIRI, pmdco.PMD_0060000, processPlanIdentifierValueIRI)) # Process plan identifier PMD:has value specification process plan identifier value
  
  # Advanced process description
  # In this section, the process of tensile testing is disassembled into individual parts, allowing a more precise process and subprocess description.

  # No. 1
  preMeasuringIRI = URIRef(experimentIRI + "_process_preMeasuring")
  g.add((preMeasuringIRI, a, pmdco.PMD_0001025)) # PMD:Assay (subclass of BFO:process)
  g.add((preMeasuringIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((processIRI, base.PMD_0060002, preMeasuringIRI)) # PMD:(entire) process starts with
  
  # No. 2
  mountingIRI = URIRef(experimentIRI + "_process_mounting")
  g.add((mountingIRI, a, pmdco.PMD_0000524)) # PMD:assembling process
  g.add((mountingIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((mountingIRI, obo.BFO_0000062, preMeasuringIRI)) # Mounting process is BFO:preceded by preMeasuring process

  # No. 3
  gripsAlignmentIRI = URIRef(experimentIRI + "_process_gripsAlignment")
  g.add((gripsAlignmentIRI, a, pmdco.PMD_0000524)) # PMD:assembling process
  g.add((gripsAlignmentIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((gripsAlignmentIRI, obo.BFO_0000062, mountingIRI)) # Grips Alignment process is BFO:preceded by Mounting process

  # No. 4
  tensileTestISOIRI = URIRef(experimentIRI + "_process_tensileTestISO")
  g.add((tensileTestISOIRI, a, pmdco.PMD_0000017)) # PMD:tensile testing process
  # g.add((tensileTestISOIRI, a, base.TensileTest)) # ISO Tensile test process may be additionally defined in TTO someday (!?), to explicitly represent the standard test operation / procedure
  g.add((tensileTestISOIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((tensileTestISOIRI, obo.BFO_0000062, gripsAlignmentIRI)) # ISO Tensile Test process is BFO:preceded by Grips Alignment process

  # No. 5
  demountingIRI = URIRef(experimentIRI + "_process_demounting")
  g.add((demountingIRI, a, pmdco.PMD_0000524)) # PMD:assembling process
  g.add((demountingIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((demountingIRI, obo.BFO_0000062, tensileTestISOIRI)) # Demounting process is BFO:preceded by ISO Tensile Test process

  # No. 6
  postMeasuringIRI = URIRef(experimentIRI + "_process_postMeasuring")
  g.add((postMeasuringIRI, a, pmdco.PMD_0001025)) # PMD:Assay (subclass of BFO:process)
  g.add((postMeasuringIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((postMeasuringIRI, obo.BFO_0000062, demountingIRI)) # Post Measuring process is BFO:preceded by Demounting process

  # No. 7
  analysisIRI = URIRef(experimentIRI + "_process_analysis")
  g.add((analysisIRI, a, pmdco.PMD_0000583)) # PMD:Computing process which is a subclass of BFO:process
  g.add((analysisIRI, obo.BFO_0000132, processIRI)) # BFO:Occurrent part of (entire) process
  g.add((analysisIRI, obo.BFO_0000062, postMeasuringIRI)) # Analysis process is BFO:preceded by Post Measuring process

  
  # Object / test piece description
  # Test piece as input prior to tensile test
  testpieceIRI = URIRef(prefix + f"testpiece/{testpiece_id}")
  g.add((testpieceIRI, a, base.TTO_0000055)) # TTO:Test piece as subclass of BFO:object, subclass of BFO:material entity
  testpieceID_IRI = URIRef(prefix + f"testpiece_name/{testpiece_id}")
  g.add((testpieceID_IRI, a, pmdco.PMD_0000017)) # PMD:identifier
  testpieceIDvalue_IRI = URIRef(prefix + f"testpiece_name/value/{testpiece_id}")
  g.add((testpieceIDvalue_IRI, a, obo.OBI_0001933)) # OBI:value specification
  
  g.add((testpieceIDvalue_IRI, pmdco.PMD_0000006, Literal(testpiece_id, datatype=XSD.string))) # PMD:has value
  g.add((testpieceID_IRI, pmdco.PMD_0060000, testpieceIDvalue_IRI)) # testpiece ID PMD:has value specification testpiece ID value
  g.add((testpieceID_IRI, obo.IAO_0000219, testpieceIRI)) # testpiece ID IAO:denotes testpiece

  # Test piece(s) as output after tensile test (typically, 2 fractured parts will be formed)
  # Test piece after test 1
  testpieceAfterTest1IRI = URIRef(testpieceIRI  + "_afterTest_1")
  g.add((testpieceAfterTest1IRI, a, base.TTO_0000055)) # TTO:Test piece as subclass of BFO:object, subclass of BFO:material entity
  testpieceAfterTest1ID_IRI = URIRef(testpieceID_IRI + "_afterTest_1")
  g.add((testpieceAfterTest1ID_IRI, a, pmdco.PMD_0000017)) # PMD:identifier
  testpieceAfterTest1IDvalue_IRI = URIRef(testpieceIDvalue_IRI + "_afterTest_1")
  g.add((testpieceAfterTest1IDvalue_IRI, a, obo.OBI_0001933)) # OBI:value specification
  
  g.add((testpieceAfterTest1IDvalue_IRI, pmdco.PMD_0000006, Literal(testpiece_id + "_afterTest_1", datatype=XSD.string))) # PMD:has value
  g.add((testpieceAfterTest1ID_IRI, pmdco.PMD_0060000, testpieceAfterTest1IDvalue_IRI)) # testpiece ID PMD:has value specification testpiece ID value
  g.add((testpieceAfterTest1ID_IRI, obo.IAO_0000219, testpieceAfterTest1IRI)) # testpiece ID IAO:denotes testpiece

  # Test piece after test 2
  testpieceAfterTest2IRI = URIRef(testpieceIRI  + "_afterTest_2")
  g.add((testpieceAfterTest2IRI, a, base.TTO_0000055)) # TTO:Test piece as subclass of BFO:object, subclass of BFO:material entity
  testpieceAfterTest2ID_IRI = URIRef(testpieceID_IRI + "_afterTest_2")
  g.add((testpieceAfterTest2ID_IRI, a, pmdco.PMD_0000017)) # PMD:identifier
  testpieceAfterTest2IDvalue_IRI = URIRef(testpieceIDvalue_IRI + "_afterTest_2")
  g.add((testpieceAfterTest2IDvalue_IRI, a, obo.OBI_0001933)) # OBI:value specification
  
  g.add((testpieceAfterTest2IDvalue_IRI, pmdco.PMD_0000006, Literal(testpiece_id + "_afterTest_2", datatype=XSD.string))) # PMD:has value
  g.add((testpieceAfterTest2ID_IRI, pmdco.PMD_0060000, testpieceAfterTest2IDvalue_IRI)) # testpiece ID PMD:has value specification testpiece ID value
  g.add((testpieceAfterTest2ID_IRI, obo.IAO_0000219, testpieceAfterTest2IRI)) # testpiece ID IAO:denotes testpiece

  # Test piece roles
  testpieceRoleIRI = URIRef(prefix + f"testpiece/role/{testpiece_id}")
  g.add((testpieceRoleIRI, a, pmdco.PMD_0000975)) # PMD:test piece role
  testpieceAfterTest1RoleIRI = URIRef(testpieceRoleIRI  + "_afterTest_1")
  g.add((testpieceAfterTest1RoleIRI, a, pmdco.PMD_0000975)) # PMD:test piece role
  testpieceAfterTest2RoleIRI = URIRef(testpieceRoleIRI  + "_afterTest_2")
  g.add((testpieceAfterTest2RoleIRI, a, pmdco.PMD_0000975)) # PMD:test piece role
  
  g.add((testpieceIRI, obo.BFO_0000196, testpieceRoleIRI)) # testpiece BFO:bearer of testpiece role
  g.add((testpieceAfterTest1IRI, obo.BFO_0000196, testpieceAfterTest1RoleIRI)) # Test piece after test 1 BFO:bearer of testpiece role
  g.add((testpieceAfterTest2IRI, obo.BFO_0000196, testpieceAfterTest2RoleIRI)) # Test piece after test 2 BFO:bearer of testpiece role
  
  g.add((processIRI, obo.BFO_0000055, testpieceRoleIRI)) # process (tensile test) BFO:realizes testpiece role for test piece
  g.add((processIRI, obo.BFO_0000055, testpieceAfterTest1RoleIRI)) # process (tensile test) BFO:realizes testpiece role for test piece after test 1
  g.add((processIRI, obo.BFO_0000055, testpieceAfterTest2RoleIRI)) # process (tensile test) BFO:realizes testpiece role for test piece after test 2
  
  
  # Process - Test Piece Relations
  g.add((processIRI, pmdco.PMD_0000015, testpieceIRI)) # Process (Tensile Test) PMD:has input testpiece
  g.add((processIRI, pmdco.PMD_0000016, testpieceAfterTest1IRI)) # Process (Tensile Test) PMD:has output testpiece after test 1
  g.add((processIRI, pmdco.PMD_0000016, testpieceAfterTest2IRI)) # Process (Tensile Test) PMD:has output testpiece after test 2
  g.add((testpieceIRI, obo.BFO_0000178, testpieceAfterTest1IRI)) # Testpiece BFO:has continuant part Testpiece after test 1 
  g.add((testpieceIRI, obo.BFO_0000178, testpieceAfterTest2IRI)) # Testpiece BFO:has continuant part Testpiece after test 2


  # The next section contains some material / test piece information.
  # Material designation and material specification / description
  materialIRI = URIRef(testpieceIRI + "_material")
  g.add((materialIRI, a, pmdco.PMD_0020096)) # is a PMD:Steel as subclass of PMD:ferrous alloy --> PMD:metal --> BFO:material entity
  g.add((testpieceIRI, pmdco.PMD_0020005, materialIRI)) # Test piece (material entity) PMD:consists of PMD:Steel (material entity)
  g.add((testpieceAfterTest1IRI, pmdco.PMD_0020005, materialIRI)) # Test piece after fracture 1 (material entity) PMD:consists of PMD:Steel (material entity)
  g.add((testpieceAfterTest2IRI, pmdco.PMD_0020005, materialIRI)) # Test piece after fracture 2 (material entity) PMD:consists of PMD:Steel (material entity)
  
  materialIdentifierIRI = URIRef(testpieceIRI + "_material_identifier")
  g.add((materialIdentifierIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier as subclass of IAO:information content entity
  g.add((materialIdentifierIRI, obo.IAO_0000219, materialIRI)) # (material) identifier IAO:denotes material
  materialIdentifierValueIRI = URIRef(testpieceIRI + "_material_identifier_value")
  g.add((materialIdentifierValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity  
  g.add((materialIdentifierValueIRI, pmdco.PMD_0000006, Literal(materialDescription, datatype=XSD.string))) # Material identifier value PMD:has value Literal
  g.add((materialIdentifierIRI, pmdco.PMD_0060000, materialIdentifierValueIRI)) # Material identifier PMD:has value specification Material identifier value
  
  # Specimen / test piece shape
  specimenShapeIRI = URIRef(testpieceIRI + "_specimen_shape")
  g.add((specimenShapeIRI, a, base.TTO_0000048)) # is a TTO:shape 3D as subclass of BFO:quality
  specimenShapeValueSpecificationIRI = URIRef(testpieceIRI + "_specimen_shape_value_specification")
  g.add((specimenShapeValueSpecificationIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
  g.add((specimenShapeValueSpecificationIRI, pmdco.PMD_0000006, Literal(specimenShape, datatype=XSD.string))) # Specimen shape value specification PMD:has value Literal (string)
  
  g.add((testpieceIRI, obo.BFO_0000196, specimenShapeIRI)) # Test piece BFO:bearer of specimen Shape
  g.add((specimenShapeIRI, pmdco.PMD_0060000, specimenShapeValueSpecificationIRI)) # Specimen shape PMD:has value specification Specimen shape value specification
  g.add((specimenShapeValueSpecificationIRI, obo.IAO_0000136, testpieceIRI)) # Specimen shape value specification IAO:is about Test piece (material entity)
  
  # Test piece specification: Description of rolling direction
  rollingDirectionInformationIRI = URIRef(testpieceIRI + "_rolling_direction_information")
  g.add((rollingDirectionInformationIRI, a, pmdco.PMD_0060007)) # is a PMD:material specification as subclass of IAO:information content entity
  g.add((rollingDirectionInformationIRI, obo.IAO_0000136, testpieceIRI)) # material specification IAO:is about test piece
  rollingDirectionInformationValueIRI = URIRef(testpieceIRI + "_rolling_direction_information_value")
  g.add((rollingDirectionInformationValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity  
  g.add((rollingDirectionInformationValueIRI, pmdco.PMD_0000006, Literal(rollingDirection, datatype=XSD.string))) # Rolling Direction Information Value PMD:has value Literal
  g.add((rollingDirectionInformationIRI, pmdco.PMD_0060000, rollingDirectionInformationValueIRI)) # Rolling Direction Information PMD:has value specification Rolling Direction Information Value
  
  # The next section contains some primary data.

  # Raw data / dataset
  # Using CSVW ontology vocabulary for raw / primary data set description
  datasetIRI = URIRef(experimentIRI + "resource/" + testpiece_id + "_dataset")
  g.add((datasetIRI, a, csvw.Table)) # is a csvw:Table
  g.add((datasetIRI, a, obo.IAO_0000109)) # is an iao:measurement datum as subclass of IAO:data item (IAO:information content entity)
  g.add((datasetIRI, DC.title, Literal(f"process/{process_id}" + f"testpiece_id/{testpiece_id}", datatype=XSD.string)))
  g.add((datasetIRI, csvw.url, Literal(raw_data + f"{testpiece_id}" + ".csv", datatype=XSD.string)))
  g.add((datasetIRI, DC.format, Literal("text/csv", datatype=XSD.string)))
  g.add((processIRI, pmdco.PMD_0000016, datasetIRI)) # Entire Tensile Test Process PMD:has output dataset / measurement datum
  g.add((tensileTestISOIRI, pmdco.PMD_0000016, datasetIRI)) # ISO Tensile Test Process (subprocess) PMD:has output dataset / measurement datum

  schemaIRI = URIRef(experimentIRI + testpiece_id + "_schema")
  g.add((schemaIRI, a, csvw.Schema))
  g.add((datasetIRI, csvw.tableSchema, schemaIRI))

  BNode1 = BNode()
  BNode2 = BNode()
  BNode3 = BNode()
  BNode4 = BNode()

  column1 = BNode()
  g.add((column1, a, csvw.Column))
  # g.add((column1, a, time.Duration)) # is a time:Duration as subclass of time:TemporalDuration (owl:Class)
  g.add((column1, a, obo.BFO_0000008)) # is a BFO:temporal region as subclass of BFO:occurrent
  g.add((column1, pmdco.PMD_0000020, unit.SEC)) # Column PMD:has measurement unit label 
  g.add((column1, csvw.name, Literal("Time", datatype=XSD.string)))
  g.add((column1, DC.title, Literal("Time", datatype=XSD.string)))
  g.add((column1, csvw.datatype, Literal(1, datatype=XSD.float))) # here, also xsd:duration may be used; although, a decimal is given in the data!
  g.add((column1, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column2 = BNode()
  g.add((column2, a, csvw.Column))
  g.add((column2, a, base.TTO_0000013)) # is a TTO:crosshead separation
  g.add((column2, pmdco.PMD_0000020, unit.MilliM)) # Column PMD:has measurement unit label
  g.add((column2, csvw.name, Literal("Crosshead travel", datatype=XSD.string)))
  g.add((column2, DC.title, Literal("Crosshead Separation", datatype=XSD.string)))
  g.add((column2, csvw.datatype, Literal(2, datatype=XSD.float)))
  g.add((column2, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column3 = BNode()
  g.add((column3, a, csvw.Column))
  g.add((column3, a, pmdco.PMD_0020200)) # is a PMD:force
  g.add((column3, pmdco.PMD_0000020, unit.KiloN)) # Column PMD:has measurement unit label
  g.add((column3, csvw.name, Literal("Force", datatype=XSD.string)))
  g.add((column3, DC.title, Literal("Force", datatype=XSD.string)))
  g.add((column3, csvw.datatype, Literal(3, datatype=XSD.float)))
  g.add((column3, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column4 = BNode()
  g.add((column4, a, csvw.Column))
  g.add((column4, a, base.TTO_0000034)) # is a TTO:Percentage Extension
  g.add((column4, pmdco.PMD_0000020, unit.PERCENT)) # Column PMD:has measurement unit label 
  g.add((column4, csvw.name, Literal("Strain", datatype=XSD.string)))
  g.add((column4, DC.title, Literal("Percentage Extension", datatype=XSD.string)))
  g.add((column4, csvw.datatype, Literal(4, datatype=XSD.float)))
  g.add((column4, csvw.headerCount, Literal(2, datatype=XSD.float)))

  g.add((schemaIRI, csvw.column, BNode1))
  g.add((BNode1, RDF.first, column1))
  g.add((BNode1, RDF.rest, BNode2))
  g.add((BNode2, RDF.first, column2))
  g.add((BNode2, RDF.rest, BNode3))
  g.add((BNode3, RDF.first, column3))
  g.add((BNode3, RDF.rest, BNode4))
  g.add((BNode4, RDF.first, column4))
  g.add((BNode4, RDF.rest, RDF.nil))

  # Rest of primary data, mainly being represented as BFO:qualities in TTO
  # Original thicknesses a1, a2, a3 as output of preMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="thickness_a1", value=a1, unit=unit.MilliM, quality_class=base.TTO_0000029, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="thickness_a2", value=a2, unit=unit.MilliM, quality_class=base.TTO_0000029, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="thickness_a3", value=a3, unit=unit.MilliM, quality_class=base.TTO_0000029, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  # Original widths b1, b2, b3 as output of preMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="width_b1", value=b1, unit=unit.MilliM, quality_class=base.TTO_0000030, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="width_b2", value=b2, unit=unit.MilliM, quality_class=base.TTO_0000030, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="width_b3", value=b3, unit=unit.MilliM, quality_class=base.TTO_0000030, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  # Cross section areas s1, s2, s3, S0 as output of preMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="crossSectionArea_s1", value=s1, unit=unit.MilliM2, quality_class=base.TTO_0000012, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="crossSectionArea_s2", value=s2, unit=unit.MilliM2, quality_class=base.TTO_0000012, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="crossSectionArea_s3", value=s3, unit=unit.MilliM2, quality_class=base.TTO_0000012, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="crossSectionArea_s0", value=S0, unit=unit.MilliM2, quality_class=base.TTO_0000012, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  # Original gauge length L0 as output of preMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="originalGaugeLength_L0", value=L0, unit=unit.MilliM, quality_class=base.TTO_0000012, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=preMeasuringIRI)
  # Final gauge length after fracture Lu as output of postMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="finalGaugeLengthAfterFracture_Lu", value=Lu, unit=unit.MilliM, quality_class=base.TTO_0000018, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=postMeasuringIRI)
  # Thickness after fracture au as output of postMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="thicknessAfterFracture_au", value=au, unit=unit.MilliM, quality_class=base.TTO_0000056, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=postMeasuringIRI)
  # Width after fracture bu as output of postMeasuring process
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="widthAfterFracture_bu", value=bu, unit=unit.MilliM, quality_class=base.TTO_0000056, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=postMeasuringIRI)
  # Cross section area after fracture Su as output of analysis process (is calculated therein)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="crossSectionArea_Su", value=Su, unit=unit.MilliM, quality_class=base.TTO_0000012, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)

  # The next section contains some secondary data.
  # Proof strength plastic extension Rp02 as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="proofStrength_Rp02", value=Rp02, unit=unit.MegaPa, quality_class=base.TTO_0000047, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Upper yield strength ReH as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="upperYieldStrength_ReH", value=ReH, unit=unit.MegaPa, quality_class=base.TTO_0000059, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Maximum force Fm as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="maximumForce_Fm", value=Fm, unit=unit.kiloN, quality_class=base.TTO_0000023, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Tensile Strength Rm as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="tensileStrength_Rm", value=Rm, unit=unit.MegaPa, quality_class=base.TTO_0000053, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Modulus of elasticity E as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="modulusOfElasticity_E", value=E, unit=unit.GigaPa, quality_class=pmdco.PMD_0000618, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Slope of the elastic part as output of analysis process (is calculated / computed within this process); in this case, it is the same value as "Modulus of elasticity E"
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="modulusOfElasticity_E", value=E, unit=None, quality_class=base.TTO_0000050, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Percentage Elongation After Fracture A as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="percentageElongationAfterFracture_A", value=A, unit=unit.PERCENT, quality_class=base.TTO_0000033, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  # Percentage Reduction Of Area Z as output of analysis process (is calculated / computed within this process)
  add_scalar_value_measurement(g=g, experimentIRI=experimentIRI, measurement_name="percentageReductionOfArea_Z", value=Z, unit=unit.PERCENT, quality_class=base.TTO_0000038, testpieceIRI=testpieceIRI, processIRI=processIRI, processPartIRI=analysisIRI)
  
  # The next section contains metadata concerning the process itself.
  # Environmental Temperature as metadata of the process
  add_scalar_value_measurement_metadata(g=g, experimentIRI=experimentIRI, measurement_name="environmentalTemperature", value=environmentalTemperature, unit=unit.DEG_C, quality_class=pmdco.PMD_0000967, processIRI=processIRI)
  
  # Date of measurement as a temporal instant
  dateIRI = URIRef(experimentIRI + "_date")
  g.add((dateIRI, a, obo.BFO_0000203)) # is a BFO:temporal instant (Definition: A temporal entity with zero extent or duration)
  g.add((processIRI, obo.BFO_0000108, dateIRI)) # Process BFO:exists at date
  
  dateValueSpecificationIRI = URIRef(experimentIRI + "_date_specification_value")
  g.add((dateValueSpecificationIRI, a, obo.BFO_0001933)) # is a BFO:value specification
  g.add((dateValueSpecificationIRI, pmdco.PMD_0060001, dateIRI)) # value specification PMD:specifies value of date (quality) defined above
  g.add((dateValueSpecificationIRI, pmdco.PMD_0000006, Literal(date, datatype=XSD.date))) # value specification PMD:has value date (variable)

  # Project and Funding information
  projectIRI = add_project(g=g, experimentIRI=experimentIRI, processIRI=processIRI, project_literal=project)
  # Funder (BMBF)
  add_funder(g=g, experimentIRI=experimentIRI, projectIRI=projectIRI, funder_literal_de="Bundesministerium für Bildung und Forschung (BMBF)@de", funder_literal_en="Federal Ministry of Education and Research@en", funder_number_literal="Q492234")
  # Funding Project
  fundingProjectIRI = add_funding_project(g=g, experimentIRI=experimentIRI, projectIRI=projectIRI, funding_project_literal="13XP5094")

  # Operator
  operatorRoleIRI = URIRef(experimentIRI + "_operator_role")
  g.add((operatorRoleIRI, a, pmdco.PMD_0000873)) # is a pmdco:operator role as subclass of BFO:role (BFO:realizable entity)
  g.add((operatorRoleIRI, obo.BFO_0000054, processIRI)) # Operator Role BFO:has realization (in) (tensile testing) process
    
  operatorIRI = URIRef(experimentIRI + "_operator")
  g.add((operatorIRI, a, pmdco.PMD_0000881)) # is a PMD:person as subclass of BFO:object (material entity)
  g.add((operatorIRI, obo.BFO_0000196, operatorRoleIRI)) # Person BFO:bearer of operator role
  g.add((processIRI, obo.BFO_0000057, operatorIRI)) # (Tensile Test) Process BFO:has participant Person
  
  operatorNameIRI = URIRef(experimentIRI + "_operator_name")
  g.add((operatorNameIRI, a, obo.BFO_0001933)) # is a BFO:value specification
  g.add((operatorNameIRI, pmdco.PMD_0060001, operatorIRI)) # value specification PMD:specifies value of operator / person (object) defined above
  g.add((operatorNameIRI, pmdco.PMD_0000006, Literal(operator, datatype=XSD.string))) # value specification PMD:has value operator (variable)
  
  # Note
  noteIRI = URIRef(experimentIRI + "_note")
  g.add((noteIRI, a, pmdco.PMD_0060009)) # is a PMD:specification datum as subclass of data item (information content entity)
  g.add((noteIRI, obo.IAO_0000219, processIRI)) # ("Note") specification datum IAO:denotes (Tensile Test) Process
  
  noteValueSpecification = URIRef(experimentIRI + "_note_value_specification")
  g.add((noteValueSpecification, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of information content entity
  g.add((noteIRI, pmdco.PMD_0060000, noteValueSpecification)) # ("Note") specification datum PMD:has value specification note value specification
  g.add((noteValueSpecification, pmdco.PMD_0000006, Literal(note, datatype=XSD.string))) # note value specification PMD:has value note (varibable)
  
  # The next section contains metadata on location, address, institute, etc.
  # 'Static' data such as institution and location get a static URI (the same URI / instance for every process regarded)
  # (there is only one of the explicitely specified institution and location that is referred to, respectively)
  institutionIRI = URIRef(experimentIRI + "_institute")
  g.add((institutionIRI, a, pmdco.PMD_0000879)) # is a PMD:organization (material entity)
  g.add((processIRI, obo.BFO_0000057, institutionIRI)) # (Tensile Test) Process BFO:has participant institution
  
  labIRI = URIRef(experimentIRI + "_laboratory")
  g.add((labIRI, a, pmdco.PMD_0000814)) # is a PMD:laboratory (immaterial entity)
  g.add((labIRI, obo.BFO_0000171, institutionIRI)) # Laboratory BFO:located in institution  
  
  labRoleIRI = URIRef(experimentIRI + "_laboratory_role")
  g.add((labRoleIRI, a, pmdco.PMD_0000815)) # is a PMD:laboratory role (BFO:role)
  g.add((labIRI, obo.BFO_0000196, labRoleIRI)) # Lab (is) BFO:bearer of PMD:laboratory role (BFO:role)
  g.add((labRoleIRI, obo.BFO_0000054, processIRI)) # Laboratory Role is bfo:realized in (tensile testing) process
    
  # Institution identifiers
  institutionNameIRI = URIRef(experimentIRI + "_institute_ID_name")
  g.add((institutionNameIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
  g.add((institutionNameIRI, obo.IAO_0000219, institutionIRI)) # Insitution Name (identifier) iao:denotes institution

  institutionNameValueIRI = URIRef(experimentIRI + "_institution_name")
  g.add((institutionNameValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
  g.add((institutionNameValueIRI, pmdco.PMD_0000006, Literal(institute, datatype=XSD.string))) # Institution identifier value PMD:has value institute (variable)
  g.add((institutionNameIRI, pmdco.PMD_0060000, institutionNameValueIRI)) # Institution name identifier PMD:has value specification institution identifier value

  institutionAddressIRI = URIRef(experimentIRI + "_institute_ID_address")
  g.add((institutionAddressIRI, a, pmdco.PMD_0000017)) # is a PMD:identifier
  g.add((institutionAddressIRI, obo.IAO_0000219, institutionIRI)) # Insitution address (identifier) iao:denotes institution

  institutionAddressValueIRI = URIRef(experimentIRI + "_institution_address")
  g.add((institutionAddressValueIRI, a, obo.OBI_0001933)) # is a OBI:value specification as subclass of IAO:information content entity
  g.add((institutionAddressValueIRI, pmdco.PMD_0000006, Literal(f"{address}" + " " + f"{location}", datatype=XSD.string))) # Institution address identifier value PMD:has value address + location (variables)
  g.add((institutionAddressIRI, pmdco.PMD_0060000, institutionAddressValueIRI)) # Institution address identifier PMD:has value specification institution identifier value

  # The next section contains metadata concerning machines, components (processing nodes) used.
  # 'Static' data such as manufacturers and serial numbers get a static URI (the same URI / instance for every process regarded)
  # Caution: If several machines were used, the machineIRI will have to be changed:
  # Then, it will have to contain the machine designation in the IRI (cf. e.g. institute)

  # Tensile Testing Machine
  add_test_device(g=g, experimentIRI=experimentIRI, processIRI=processIRI, deviceDenotation="tensile_testing_machine", deviceClass=pmdco.PMD_0000973, functionClass=pmdco.PMD_0000976, processPart1IRI=tensileTestISOIRI, processPart2IRI=None, deviceName=machineName, deviceManufacturer=machineManufacturer, deviceType=machineType, deviceSerialNumber=machineSerialNumber, deviceStandard=machineStandard)
  add_test_device(g=g, experimentIRI=experimentIRI, processIRI=processIRI, deviceDenotation="extensometer", deviceClass=pmdco.PMD_0000636, functionClass=pmdco.PMD_0000610, processPart1IRI=tensileTestISOIRI, processPart2IRI=None, deviceName=extensometerName, deviceManufacturer=None, deviceType=None, deviceSerialNumber=extensometerSerialNumber, deviceStandard=extensometerStandard)

  # Extensometer Gauge Length
  extensometerIRI = experimentIRI + f"_extensometer"
  
  extensometerGaugeLengthIRI_value_specification = URIRef(experimentIRI + "_extensometerGaugeLength_value_specification")
  g.add((extensometerGaugeLengthIRI_value_specification, a, pmdco.PMD_0000023)) # is a PMD:scalar value specification
  g.add((extensometerGaugeLengthIRI_value_specification, pmdco.PMD_0000006, Literal(extensometerGaugeLength, datatype=XSD.float))) # Extensometer gauge length value specification PMD:has value extensometerGaugeLength (variable)
  g.add((extensometerGaugeLengthIRI_value_specification, pmdco.PMD_0000020, unit.MilliM)) # Extensometer gauge length value specification PMD:has measurement unit label MilliM (QUDT)
  
  extensometerGaugeLengthIRI_measurement_datum = URIRef(experimentIRI + "_extensometerGaugeLength_measurement_datum")
  g.add((extensometerGaugeLengthIRI_measurement_datum, a, pmdco.PMD_0000023)) # is a PMD:scalar measurement datum
  g.add((extensometerGaugeLengthIRI_measurement_datum, pmdco.PMD_0060000, extensometerGaugeLengthIRI_value_specification)) # Extensometer gauge length measurement datum PMD:has value specification Extensometer gauge length value specification

  extensometerGaugeLengthIRI_quality = URIRef(experimentIRI + "_extensometer_gauge_length")
  g.add((extensometerGaugeLengthIRI_quality, a, base.TTO_0000017)) # is a Extensometer Gauge Length as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((extensometerGaugeLengthIRI_quality, obo.IAO_0000417, extensometerGaugeLengthIRI_measurement_datum)) # Extensometer gauge length quality IAO:is quality measured as Extensometer gauge length
  g.add((extensometerGaugeLengthIRI_measurement_datum, obo.IAO_0000136, extensometerIRI)) # Extensometer gauge length measurement datum IAO:is about Extensometer
  g.add((extensometerIRI, obo.BFO_0000196, extensometerGaugeLengthIRI_quality)) # Extensometer BFO:bearer of Extensometer gauge length quality

  # Load Cell
  add_test_device(g=g, experimentIRI=experimentIRI, processIRI=processIRI, deviceDenotation="load_cell", deviceClass=pmdco.PMD_0000820, functionClass=pmdco.PMD_0000644, processPart1IRI=tensileTestISOIRI, processPart2IRI=None, deviceName=None, deviceManufacturer=None, deviceType=None, deviceSerialNumber=None, deviceStandard=None)
  
  # Load Cell Specifications
  loadCellIRI = experimentIRI + f"_load_cell"
  
  loadCellMaxForceIRI_value_specification = URIRef(experimentIRI + "_loadCellMaxForce_value_specification")
  g.add((loadCellMaxForceIRI_value_specification, a, pmdco.PMD_0000023)) # is a PMD:scalar value specification
  g.add((loadCellMaxForceIRI_value_specification, pmdco.PMD_0000006, Literal(loadCellMaxForce, datatype=XSD.float))) # Load cell max force value specification PMD:has value loadCellMaxForce (variable)
  g.add((loadCellMaxForceIRI_value_specification, pmdco.PMD_0000020, unit.N)) # Load cell max force value specification PMD:has measurement unit label N (QUDT)
  
  loadCellMaxForceIRI_measurement_datum = URIRef(experimentIRI + "_loadCellMaxForce_measurement_datum")
  g.add((loadCellMaxForceIRI_measurement_datum, a, pmdco.PMD_0000023)) # is a PMD:scalar measurement datum
  g.add((loadCellMaxForceIRI_measurement_datum, pmdco.PMD_0060000, loadCellMaxForceIRI_value_specification)) # Load cell max force measurement datum PMD:has value specification Load cell max force value specification

  loadCellMaxForceRealizableEntityIRI = URIRef(experimentIRI + "_load_cell_max_force_realizable_entity")
  g.add((loadCellMaxForceRealizableEntityIRI, a, base.TTO_0000001)) # is a load cell maximum force as subclass of pmdco:force, being subclass to BFO:realizable entity
  g.add((loadCellMaxForceRealizableEntityIRI, obo.IAO_0000417, loadCellMaxForceIRI_measurement_datum)) # Load cell max force realizable entity IAO:is quality measured as Load cell max force measurement datum
  g.add((loadCellMaxForceIRI_measurement_datum, obo.IAO_0000136, loadCellIRI)) # Load cell max force measurement datum IAO:is about load cell
  g.add((loadCellIRI, obo.BFO_0000196, extensometerGaugeLengthIRI_quality)) # Load cell BFO:bearer of Load cell max force realizable entity

  # Micrometer Gauge
  add_test_device(g=g, experimentIRI=experimentIRI, processIRI=processIRI, deviceDenotation="micormeter_gauge", deviceClass=pmdco.PMD_0000854, functionClass=pmdco.PMD_0000610, processPart1IRI=preMeasuringIRI, processPart2IRI=postMeasuringIRI, deviceName=None, deviceManufacturer=micrometerGaugeManufacturer, deviceType=None, deviceSerialNumber=None, deviceStandard=None)
  # Caliper
  add_test_device(g=g, experimentIRI=experimentIRI, processIRI=processIRI, deviceDenotation="caliper", deviceClass=pmdco.PMD_0000544, functionClass=pmdco.PMD_0000610, processPart1IRI=preMeasuringIRI, processPart2IRI=postMeasuringIRI, deviceName=None, deviceManufacturer=caliperManufacturer, deviceType=None, deviceSerialNumber=caliperSerialNumber, deviceStandard=None)
  
  # Strain Rate
  machineIRI = experimentIRI + f"tensile_testing_machine"
  
  strainRateIRI_value_specification = URIRef(experimentIRI + "_strain_rate_value_specification")
  g.add((strainRateIRI_value_specification, a, pmdco.PMD_0000022)) # is a PMD:scalar value specification
  g.add((strainRateIRI_value_specification, pmdco.PMD_0000006, Literal(strainRate, datatype=XSD.float))) # Strain rate value specification PMD:has value strainRate (variable)

  strainRateQualityIRI = URIRef(experimentIRI + "_strain_rate_quality")
  g.add((strainRateQualityIRI, a, base.TTO_0000051)) # is a TTO:StrainRate as subclass of TTO:Movement quality --> quality
  g.add((strainRateQualityIRI, pmdco.PMD_0060000, strainRateIRI_value_specification)) # Strain Rate Quality PMD:has value specification strainRateIRI_value_specification
  g.add((machineIRI, obo.BFO_0000196, strainRateQualityIRI)) # Tensile Testing Machine BFO:bearer of Strain Rate Quality
  
  strainRateProcessCharacteristicIRI = URIRef(experimentIRI + "_strain_rate_process_characteristic")
  g.add((strainRateProcessCharacteristicIRI, a, pmdco.PMD_0000008)) # is a PMD:process characteristic as subclass of BFO:occurrent
  g.add((processIRI, pmdco.PMD_0000009, strainRateProcessCharacteristicIRI)) # (Tensile test) process PMD:has process characteristic strain rate process characteristic
  g.add((tensileTestISOIRI, pmdco.PMD_0000009, strainRateProcessCharacteristicIRI)) # (ISO Tensile test) process (subprocess) PMD:has process characteristic strain rate process characteristic
  
  # g.add((strainRateProcessCharacteristicIRI, pmdco.PMD_???, strainRateQualityIRI)) # strain rate process characteristic PMD:refers to strain rate quality

  # Transition Point
  transitionPointIRI_value_specification = URIRef(experimentIRI + "_transition_point_value_specification")
  g.add((transitionPointIRI_value_specification, a, pmdco.PMD_0000022)) # is a PMD:scalar value specification
  g.add((transitionPointIRI_value_specification, pmdco.PMD_0000006, Literal(transitionPoint, datatype=XSD.float))) # Strain rate value specification PMD:has value strainRate (variable)
  g.add((transitionPointIRI_value_specification, pmdco.PMD_0000020, unit.PERCENT)) # Transition point value specification PMD:has measurement unit label unit:PERCENT (QUDT)

  transitionPointICEIRI = URIRef(experimentIRI + "_transition_point_realizable_entity")
  g.add((transitionPointICEIRI, a, base.TTO_0000058)) # is a TTO:Transition point testing rate as subclass of information content entity
  g.add((transitionPointICEIRI, pmdco.PMD_0060000, transitionPointIRI_value_specification)) # Transition point testing rate information content entity PMD:has value specification transitionPointIRI_value_specification
  g.add((transitionPointICEIRI, obo.IAO_0000136, machineIRI)) # Transition point testing rate information content entity IAO:is about Tensile Testing Machine
  
  transitionPointProcessCharacteristicIRI = URIRef(experimentIRI + "_transition_point_process_characteristic")
  g.add((transitionPointProcessCharacteristicIRI, a, pmdco.PMD_0000008)) # is a PMD:process characteristic as subclass of BFO:occurrent
  g.add((processIRI, pmdco.PMD_0000009, transitionPointProcessCharacteristicIRI)) # (Tensile test) process PMD:has process characteristic transition point process characteristic
  g.add((tensileTestISOIRI, pmdco.PMD_0000009, transitionPointProcessCharacteristicIRI)) # (ISO Tensile test) process (subprocess) PMD:has process characteristic transition point process characteristic
  
  # g.add((transitionPointProcessCharacteristicIRI, pmdco.PMD_???, transitionPointICEIRI)) # transition point process characteristic PMD:refers to transition point information content entity

# Graph Serialization

In [20]:
# Graph g is serialized in two different data formats / notations, Turtle (ttl) and RDF
g.serialize("S355_data_tto.ttl", format="ttl")
g.serialize("S355_data_tto.rdf", format='application/rdf+xml', encoding="utf-8-sig")

# Instead of serializing the data to be stored in the working directory, 
# the files could alternatively be stored in another repository or directly
# uploaded to a triple store.

<Graph identifier=N146b474f1abf4746849ca2c94fe97ec0 (<class 'rdflib.graph.Graph'>)>

# Information concerning Namespaces and Concepts

As some ontologies concepts of which are used within this script do not use IRIs that can directly be interpreted by humans (humans will have to read the labels provided), the corresponding labels are given, here, for the most important concepts frequently used in this script or other interesting concepts that may not be used in this script. 

### Basic Formal Ontology (BFO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:BFO_0000063](http://purl.obolibrary.org/obo/BFO_0000063)       | precedes         | Object Property |
| [obo:BFO_0000055](http://purl.obolibrary.org/obo/BFO_0000055)       | realizes         | Object Property |
| [obo:BFO_0000054](http://purl.obolibrary.org/obo/BFO_0000054)       | realized in         | Object Property |
| [obo:BFO_0000051](http://purl.obolibrary.org/obo/BFO_0000051)       | has part         | Object Property |


### Information Artifact Ontology (IAO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:IAO_0020000](http://purl.obolibrary.org/obo/IAO_0020000)       | identifier         | Class |
| [obo:IAO_0000032](http://purl.obolibrary.org/obo/IAO_0000032)       | scalar measurement datum         | Class |
| [obo:IAO_0000003](http://purl.obolibrary.org/obo/IAO_0000003)       | measurement unit label         | Class |
| [obo:IAO_0000027](http://purl.obolibrary.org/obo/IAO_0000027)       | data item         | Class |
| [obo:IAO_0000131](http://purl.obolibrary.org/obo/IAO_0000131)       | serial number         | Class |
| [obo:IAO_0000594](http://purl.obolibrary.org/obo/IAO_0000594)       | software application         | Class |
| [obo:IAO_0000100](http://purl.obolibrary.org/obo/IAO_0000100)       | data set         | Class |
| [obo:IAO_0000416](http://purl.obolibrary.org/obo/IAO_0000416)       | time measurement datum         | Class |
| [obo:IAO_0000300](http://purl.obolibrary.org/obo/IAO_0000300)       | textual entity         | Class |
| [obo:IAO_0000422](http://purl.obolibrary.org/obo/IAO_0000422)       | postal address         | Class |
| [obo:IAO_0000578](http://purl.obolibrary.org/obo/IAO_0000578)       | centrally registered identifier         | Class |
| [obo:IAO_0000039](http://purl.obolibrary.org/obo/IAO_0000039)       | has measurement unit label         | Object Property |
| [obo:IAO_0000417](http://purl.obolibrary.org/obo/IAO_0000417)       | is quality measured as         | Object Property |
| [obo:IAO_0000136](http://purl.obolibrary.org/obo/IAO_0000136)       | is about         | Object Property |
| [obo:IAO_0000219](http://purl.obolibrary.org/obo/IAO_0000219)       | denotes         | Object Property |
| [obo:IAO_0000115](http://purl.obolibrary.org/obo/IAO_0000115)       | has textual content         | Object Property |


### Ontology for Biomedical Investigations (OBI)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:OBI_0000011](http://purl.obolibrary.org/obo/OBI_0000011)       | planned process         | Class |
| [obo:OBI_0000070](http://purl.obolibrary.org/obo/OBI_0000070)       | assay         | Class |
| [obo:OBI_0001931](http://purl.obolibrary.org/obo/OBI_0001931)       | scalar value specification         | Class |
| [obo:OBI_0000260](http://purl.obolibrary.org/obo/OBI_0000260)       | plan         | Class |
| [obo:OBI_0000104](http://purl.obolibrary.org/obo/IAO_0000104)       | plan specification        | Class |
| [obo:OBI_0000835](http://purl.obolibrary.org/obo/OBI_0000835)       | manufacturer        | Class |
| [obo:OBI_0000571](http://purl.obolibrary.org/obo/OBI_0000571)       | manufacturer role       | Class |
| [obo:OBI_0000453](http://purl.obolibrary.org/obo/OBI_0000453)       | measure function       | Class |
| [obo:OBI_0000654](http://purl.obolibrary.org/obo/OBI_0000654)       | device setting       | Class |
| [obo:OBI_0001937](http://purl.obolibrary.org/obo/OBI_0001937)       | has specified numeric value         | Datatype Property |
| [obo:OBI_0002815](http://purl.obolibrary.org/obo/OBI_0002815)       | has representation         | Datatype Property |
| [obo:OBI_0000293](http://purl.obolibrary.org/obo/OBI_0000293)       | has specified input         | Object Property |
| [obo:OBI_0000299](http://purl.obolibrary.org/obo/OBI_0000299)       | has specified output         | Object Property |
| [obo:OBI_0001927](http://purl.obolibrary.org/obo/OBI_0001927)       | specifies value of         | Object Property |
| [obo:OBI_0001938](http://purl.obolibrary.org/obo/OBI_0001938)       | has value specification         | Object Property |
| [obo:OBI_0000304](http://purl.obolibrary.org/obo/OBI_0000304)       | is_manufactured_by         | Object Property |


 ### Relation Ontology (RO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:RO_0000086](http://purl.obolibrary.org/obo/RO_0000086)       | has quality         | Object Property |
| [obo:RO_0002090](http://purl.obolibrary.org/obo/RO_0002090)       | immediately precedes         | Object Property |
| [obo:RO_0002012](http://purl.obolibrary.org/obo/RO_0002012)       | occurrent part of         | Object Property |
| [obo:RO_0002224](http://purl.obolibrary.org/obo/RO_0002224)       | starts with         | Object Property |
| [obo:RO_0001000](http://purl.obolibrary.org/obo/RO_0001000)       | derives from        | Object Property |
| [obo:RO_0000059](http://purl.obolibrary.org/obo/RO_0000059)       | concretizes        | Object Property |
| [obo:RO_0002180](http://purl.obolibrary.org/obo/RO_0002180)       | has component        | Object Property |
| [obo:RO_0000085](http://purl.obolibrary.org/obo/RO_0000085)       | has function        | Object Property |
| [obo:RO_0002215](http://purl.obolibrary.org/obo/RO_0002215)       | capable of        | Object Property |
| [obo:RO_0002608](http://purl.obolibrary.org/obo/RO_0002608)       | process has causal agent       | Object Property |
| [obo:RO_0000057](http://purl.obolibrary.org/obo/RO_0000057)       | has participant       | Object Property |
| [obo:RO_0000087](http://purl.obolibrary.org/obo/RO_0000087)       | has role       | Object Property |
| [obo:RO_0002566](http://purl.obolibrary.org/obo/RO_0002566)       | causally influences       | Object Property |


### The Statistical Methods Ontology (STATO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:STATO_0000102](http://purl.obolibrary.org/obo/STATO_0000102)       | executes         | Object Property |


### Time Ontology in OWL
*(Namespace: [time:](http://www.w3.org/2006/time#)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [time:Duration](http://www.w3.org/2006/time#Duration)       | Duration         | Class |
| [time:Instant](http://www.w3.org/2006/time#Instant)       | Instant         | Class |
| [time:hasTime](http://www.w3.org/2006/time#hasTime)       | has time         | Object Property |
| [time:inXSDDate](http://www.w3.org/2006/time#inXSDDate)       | in XSD Date         | Datatype Property |
| [time:inXSDDateTime](http://www.w3.org/2006/time#inXSDDateTime)       | in XSD Date Time         | Datatype Property |